In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("coupons_ner.csv", header=None)

In [3]:
data.columns = ['description']

In [4]:
import re 
from tqdm import tqdm
def facevalue(x):
    """
    input: a data point as x
    output: the amount we save given in description else 0
    Time Complexity: O(n) # n-number of data points we a possing
    But it will work much faster than O(n) as we use map
    """ 
    if re.search("(\d+(?:\.\d+)?\¢)", x) is not None:
        return '$'+str(int(re.search("(\d+(?:\.\d+)?\¢)", x).group(1)[:-1])/10)
    elif re.search("(\$\d+(?:\.\d+)?)|[^SAVE]\.\d+(?:\.\d+)?", x) is not None:
        return re.search("(\$\d+(?:\.\d+)?)|[^SAVE]\.\d+(?:\.\d+)?", x).group(0)
#    we can add some conditions as below capture gain of the offer but as its number only field 
#    mentioned in reference pdf we are limiting for numbers only and we will add others as 0
#     elif 'BUY ONE GET ONE FREE' in x:
#         return 'BUY ONE GET ONE FREE'
#     elif 'BUY TWO GET ONE Buy ONE' in x:
#         return 'BUY TWO GET ONE Buy ONE'
#     else:
#         return "Product at good Price Already" 
    else:
        return 0
FaceValue = list(map(facevalue,tqdm(data['description'])))

100%|█████████████████████████████████████████████████████████████████████████████| 886/886 [00:00<00:00, 63270.91it/s]


In [5]:
data['FaceValue'] = FaceValue

In [6]:
data

,description,FaceValue
0,Save $2.00 ONE Downy Liquid Fabric Conditioner...,$2.00
1,Save $2.00 ONE Tide PODS OR Tide Power PODS (e...,$2.00
2,Save $2.00 ONE Tide Laundry Detergent (exclude...,$2.00
3,SAVE $1.00 ON TWO when you buy TWO BOXES (8.9 ...,$1.00
4,$3.00 OFF when you purchase any THREE (3) Pepp...,$3.00
...,...,...
881,SAVE $2.00 when you buy $7 worth of participat...,$2.00
882,SAVE $5.00 on any Gain scented purchase of $30...,$5.00
883,SAVE $1.00 when you buy any ONE (1) NEW AXE pr...,$1.00
884,BUY TWO GET ONE Buy ONE (1) CHIPS AHOY! Cookie...,9.5


In [7]:
products = pd.read_csv("product_names.csv",header=None)
products.columns = ['products']
# collect product name for corpus. here i only collected a product names from 460 rows manually and we are going to run it against entire csv
product_list_value = set()
for j in range(products.shape[0]):
    product_list_value = product_list_value.union({i.strip() for i in products.products[j].split(",")})
product_list_value = list(product_list_value)
product_list_key   = list(map(lambda x:x.lower(),product_list_value))
product_list = dict(zip(product_list_key, product_list_value))

In [8]:
product_list

{'golden ripe bananas plu': 'Golden Ripe Bananas PLU',
 'dove or dove men+care shampoo and conditioner': 'Dove or Dove Men+Care Shampoo and Conditioner',
 'tazo tea': 'Tazo Tea',
 'adult zyrtec® product': 'Adult ZYRTEC® product',
 'powder laundry detergent': 'Powder Laundry Detergent',
 'oral-b vitality': 'Oral-B Vitality',
 'pulsar': 'Pulsar',
 'oral-b glide manual floss': 'Oral-B Glide Manual Floss',
 'a to z wineworks wine': 'A to Z Wineworks wine',
 'crest®': 'CREST®',
 'degree motionsense antiperspirant or deodorant stick': 'Degree MOTIONSENSE Antiperspirant or Deodorant Stick',
 'daisy disposable': 'Daisy Disposable',
 'barista blends': 'Barista Blends',
 'tide simply pods': 'Tide Simply PODS',
 'oral-b pulsar or multi-pack adult manual toothbrush': 'Oral-B Pulsar OR Multi-pack Adult Manual Toothbrush',
 'pan roasted turkey breast': 'Pan Roasted Turkey Breast',
 'poise® pads': 'Poise® Pads',
 'nexium® 24hr': 'Nexium® 24HR',
 'tide liquid': 'Tide Liquid',
 'fresh protect': 'Fresh 

In [9]:
def find_prod(description):
    """
    input: offer description
    output: List of products in description
    Time Complexity: O(K) for single description for n description its O(K*N) 
                     where, K- number of workds in dictionary and N number of description
    """
    description = description.lower()
    temp = []
    for key in product_list:
        if key in description:
            temp.append(product_list[key])
    return temp

In [10]:
product = list(map(find_prod,data.description))

In [11]:
data['Products'] =  list(map(", ".join,product))

In [12]:
data

,description,FaceValue,Products
0,Save $2.00 ONE Downy Liquid Fabric Conditioner...,$2.00,"Fresh Protect, Downy Nature Blends, Bounce/Dow..."
1,Save $2.00 ONE Tide PODS OR Tide Power PODS (e...,$2.00,"Powder Laundry Detergent, Tide Simply PODS, Ti..."
2,Save $2.00 ONE Tide Laundry Detergent (exclude...,$2.00,"Tide Simply PODS, Tide Purclean Laundry Deterg..."
3,SAVE $1.00 ON TWO when you buy TWO BOXES (8.9 ...,$1.00,General Mills cereal
4,$3.00 OFF when you purchase any THREE (3) Pepp...,$3.00,"bread, cookies, Pepperidge Farm®, Farmhouse buns"
...,...,...,...
881,SAVE $2.00 when you buy $7 worth of participat...,$2.00,
882,SAVE $5.00 on any Gain scented purchase of $30...,$5.00,Gain
883,SAVE $1.00 when you buy any ONE (1) NEW AXE pr...,$1.00,Axe
884,BUY TWO GET ONE Buy ONE (1) CHIPS AHOY! Cookie...,9.5,"cookies, Kids, CHIPS AHOY!, larger"


In [13]:
data.to_csv("Output.csv",index = False)